In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,Orthodox,Orthodox,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.416667,99.571429,0

In [3]:
# isolating our future fights to predict
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,Orthodox,Orthodox,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.416667,99.571429,0

In [4]:
# Label encode our stances and weight class 
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df['weight_class'] = number.fit_transform(future_df['weight_class'].astype('str'))
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.416667,99.571429,0.116189,0.1394

In [5]:
# drop columns
data = future_df.drop(columns=["lev1", "winner", "red_name", "blue_name", "date", "event"])
# target = train_test_df["winner"]
# feature_names = data.columns

In [6]:
# load in our saved random forest model...
loaded_model = pickle.load(open("../saved_models/rf_up_model.sav", 'rb'))
pred = loaded_model.predict(data)
future_df["prediction"] = pred
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date,prediction
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.416667,99.571429,0.11

In [7]:
# Getting percentage certainty of prediction as an array of two outcomes, totaling 100
pcts = loaded_model.predict_proba(data)

# Looping through to get our favorite and dog in each fight
favs = []
dogs = []
for pct in pcts:
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

In [8]:
# and adding these percentages to the dataframe
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date,prediction,fav_pct,dog_pct
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.41666

In [9]:
# round off percentages to make for cleaner viewing
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date,prediction,fav_pct,dog_pct
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.731216,9.527498,7.203718,0.41666

In [10]:
# Converting percentages to a moneyline as would be seen at a fight bookie, without the house weighting
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.7312

In [11]:
# convert moneylines to int
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsAtt,red_sumTDsAtt,blue_sumSubAtt,red_sumSubAtt,blue_sumCtrlSec,red_sumCtrlSec,blue_sumKDs,red_sumKDs,blue_sumRevs,red_sumRevs,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksSeen,red_sumTotStksSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsSeen,red_sumTDsSeen,blue_sumSubEsc,red_sumSubEsc,blue_sumCtrldSec,red_sumCtrldSec,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumFightSec,red_sumFightSec,blue_sumDecWin,red_sumDecWin,blue_sumSubWin,red_sumSubWin,blue_sumKOWin,red_sumKOWin,blue_sumDecLoss,red_sumDecLoss,blue_sumSubLoss,red_sumSubLoss,blue_sumKOLoss,red_sumKOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_sumTitleFights,red_sumTitleFights,blue_winPCT,red_winPCT,red_TotStkPerMin,red_SigStkPerMin,red_TotStkAcc,red_SigStkAcc,red_StkHeadPerMin,red_StkBodyPerMin,red_StkLegPerMin,red_StkDistPerMin,red_StkClchPerMin,red_StkGndPerMin,red_TotStkAbsPerMin,red_SigStkAbsPerMin,red_StkHeadAbsPerMin,red_StkBodyAbsPerMin,red_StkLegAbsPerMin,red_StkDistAbsPerMin,red_StkClchAbsPerMin,red_StkGndAbsPerMin,red_TotStkPerMinDiff,red_SigStkPerMinDiff,red_CtrlSecPerMin,red_CtrldSecPerMin,red_CtrlDiff,red_TDAcc,red_TDDef,red_TDsPerMin,red_TDsAbsPerMin,red_KDsPer100Stk,red_reachDiff,red_heightDiff,blue_TotStkPerMin,blue_SigStkPerMin,blue_TotStkAcc,blue_SigStkAcc,blue_StkHeadPerMin,blue_StkBodyPerMin,blue_StkLegPerMin,blue_StkDistPerMin,blue_StkClchPerMin,blue_StkGndPerMin,blue_TotStkAbsPerMin,blue_SigStkAbsPerMin,blue_StkHeadAbsPerMin,blue_StkBodyAbsPerMin,blue_StkLegAbsPerMin,blue_StkDistAbsPerMin,blue_StkClchAbsPerMin,blue_StkGndAbsPerMin,blue_TotStkPerMinDiff,blue_SigStkPerMinDiff,blue_CtrlSecPerMin,blue_CtrldSecPerMin,blue_CtrlDiff,blue_TDAcc,blue_TDDef,blue_TDsPerMin,blue_TDsAbsPerMin,blue_KDsPer100Stk,blue_reachDiff,blue_heightDiff,winner,weight_class,title_fight,event,date,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,30.0,33.0,77,76,Ryan Spann,Anthony Smith,79,76,0,0,158,623,342,1206,150,536,334,1111,5,6,12,21,4,6,720,668,4,5,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,214,1362,352,1995,141,795,276,1380,6,21,14,40,1,1,410,4054,0,4,1,2,20,134,25,184,112,385,266,933,3,176,3,205,85,481,227,1030,14,77,21,111,36,137,46,181,2582,10551,2,1,1,3,2,6,0,3,0,1,1,2,5,10,1,6,0,1,0.833333,0.625000,3.542792,3.048052,0.516584,0.482448,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,7.745237,4.520898,2.189366,0.762013,1.000853,2.735286,0.437873,0.779073,-4.202445,-1.472846,3.798692,23.053739,-19.255047,0.285714,99.475000,0.034120,0.119420,0.932836,-3,-1,3.671572,3.485670,0.461988,0.449102,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,4.972889,3.276530,2.602634,0.464756,0.069713,1.975213,0.325329,0.836561,-1.301317,0.209140,16.7312

In [13]:
# load in past picks as we will add our new picks to this running tally
my_picks = pd.read_csv("picks/rolling_picks.csv")
my_picks = my_picks.drop(columns="Unnamed: 0")
my_picks.head(20)

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ryan Spann,Anthony Smith,red,0.52,0.48,-108,108
1,Devin Clark,Ion Cutelaba,blue,0.56,0.44,-127,127
2,Cameron VanCamp,Nikolas Motta,red,0.54,0.46,-117,117
3,Christos Giagos,Arman Tsarukyan,blue,0.56,0.44,-127,127
4,Antonio Arroyo,Joaquin Buckley,red,0.57,0.43,-132,132
5,Tafon Nchukwi,Mike Rodriguez,red,0.55,0.45,-122,122
6,Raquel Pennington,Pannie Kianzad,red,0.70,0.30,-233,233
7,Tony Gravely,Nate Maness,red,0.61,0.39,-156,156
8,Rong Zhu,Dakota Bush,red,0.63,0.37,-170,170
9,JP Buys,Montel Jackson,red,0.73,0.27,-270,270


In [ ]:
# dropping everything else to create a clean and simple dataframe of our future predictions 
future_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", \
                          "fav_moneyline", "dog_moneyline"]]
future_picks

In [ ]:
# This model is my prefered model so i have a running csv of all the predictions made by this model version
my_picks = pd.concat([future_picks, my_picks])
my_picks

In [ ]:
my_picks.to_csv("picks/rolling_picks.csv")

In [ ]:
# It is important to note that the moneyline is simply a straight calculation of percentage to moneyline, NOT how
# an actual moneyline would be presented at a sports book. Because bookies need to make money, and the moneylines
# they assign can be arbitrary, there will not be a straight 1-1 relationship between the favorite and dog. This is
# because bookies will account for outsides cases of draws and no contests where neither fighter is delared winner.
# It is also the case that bookies are required to make a profit, and will sometimes list BOTH fighters at negative
# money, meaning neither fighter will return more than a dollar for every dollar you bet on them to win. This is 
# seen in fights that are considered a coin flip where the "underdog" will have a moneyline of -110 (or something
# similar) and the "favorite" will be at -115 (or something similar)

# This predictive model SHOULD NOT be used for the purpose of betting without consideration of all these factors,
# and instead should be used as merely one addition datapoint to factor in if you are intending to bet on fights